In [1]:
import os
import time
import math
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.cluster import KMeans
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import torch
from carvekit.api.high import HiInterface


from PIL import Image

from carvekit.api.interface import Interface
from carvekit.ml.wrap.fba_matting import FBAMatting
from carvekit.ml.wrap.tracer_b7 import TracerUniversalB7
from carvekit.ml.wrap.u2net import U2NET
from carvekit.pipelines.postprocessing import MattingMethod
from carvekit.pipelines.preprocessing import PreprocessingStub
from carvekit.trimap.generator import TrimapGenerator

In [2]:
# manual
# Check doc strings for more information
object_type = "object"
_device='cuda' if torch.cuda.is_available() else 'cpu'

if object_type == 'hair_like':
    seg_net = U2NET(device=_device, batch_size=5,
                    input_image_size=320, fp16=True)
else:
    seg_net = TracerUniversalB7(
        device=_device, batch_size=5, input_image_size=640, fp16=True)


fba = FBAMatting(device=_device,
                 input_tensor_size=2048,
                 batch_size=1)

trimap = TrimapGenerator(prob_threshold=231,
                         kernel_size=30,
                         erosion_iters=5)

preprocessing = PreprocessingStub()

postprocessing = MattingMethod(matting_module=fba,
                               trimap_generator=trimap,
                               device=_device)
interface = Interface(pre_pipe=preprocessing,
                      post_pipe=postprocessing,
                      seg_pipe=seg_net)

# image = Image.open('SAL_images_per_60/SAL_23.jpg')
# foreground = interface([image])[0]
# foreground.save('from_code_manual.png')


In [3]:
# input video path
DIR_PATH = 'videos'
FILENAME = 'test3.mp4'
INPUT_VIDEO_PATH = os.path.join(DIR_PATH, FILENAME)
OUTPUT_PATH = 'out'

OUTPUT_FOREGROUND_VIDEO_NAME = FILENAME.split('.')[0] + '_foreground.mp4'
OUTPUT_BACKGROUND_VIDEO_NAME = FILENAME.split('.')[0] + '_background.mp4'
OUTPUT_PANORAMA_IMG_NAME = FILENAME.split('.')[0] + '_panorama.jpg'
HOLE_FILLED_BACKGROUND_VIDEO_NAME = 'hole_filled_' + FILENAME.split('.')[0] + '_background.mp4'

OUTPUT_FOREGROUND_VIDEO_PATH = os.path.join(OUTPUT_PATH, OUTPUT_FOREGROUND_VIDEO_NAME)
OUTPUT_BACKGROUND_VIDEO_PATH = os.path.join(OUTPUT_PATH, OUTPUT_BACKGROUND_VIDEO_NAME)
OUTPUT_PANORAMA_IMG_PATH = os.path.join(OUTPUT_PATH, OUTPUT_PANORAMA_IMG_NAME)
HOLE_FILLED_BACKGROUND_VIDEO_PATH = os.path.join(OUTPUT_PATH, HOLE_FILLED_BACKGROUND_VIDEO_NAME)

In [4]:
def read_frames(path):
    """
    return video frames
    """
    cap = cv.VideoCapture(path)
    if not cap.isOpened():
        raise IOError("Open video failed!")

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            break

        frames.append(frame)
        
    cap.release()
    return frames


cap = cv.VideoCapture(INPUT_VIDEO_PATH)
if not cap.isOpened():
    raise IOError("Open video failed!")

fps = int(cap.get(cv.CAP_PROP_FPS))
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
cap.release()

In [5]:
def get_foreground_labels(path, early_quit_frame_number=math.inf, early_quit_time=None):
    cap = cv.VideoCapture(path)
    if not cap.isOpened:
        raise IOError("Open video failed!")

    foreground_labels = []
    startTime = time.time()
    try:
        for _ in tqdm(range(int(min(cap.get(cv.CAP_PROP_FRAME_COUNT), early_quit_frame_number)))):
            if early_quit_time and time.time() - startTime > early_quit_time:
                break
            
            ret, frame = cap.read()
            if not ret:
                raise IOError("Read frame failed!")
                
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            image = Image.fromarray(frame, mode="RGB")
            resImg = interface([image])[0]
            foreground = np.asarray(resImg)
            foreground_label = set()
            for i in range(len(foreground)):
                for j in range(len(foreground[0])):
                    if foreground[i][j][-1] != 0:
                        foreground_label.add((i, j))
            foreground_labels.append(foreground_label)

    except KeyboardInterrupt:
        print('Interrupted!')    

    cap.release()
    cv.destroyAllWindows()
    return foreground_labels

In [ ]:
# # save foreground labels
# foreground_label_persistence_file = '{}_foreground_labels.npy'.format(FILENAME.split('.')[0])
# np.save(foreground_label_persistence_file, [list(label) for label in foreground_labels])

In [6]:
# load foreground_labels
foreground_label_persistence_file = '{}_foreground_labels.npy'.format(FILENAME.split('.')[0])
foreground_labels = np.load(foreground_label_persistence_file, allow_pickle=True)
# convert foreground_labels to list of set
foreground_labels = [set(_) for _ in foreground_labels]

In [7]:
def generate_foreground_background_videos(foreground_labels):
    if not os.path.exists('out'):
        os.makedirs('out')

    cap = cv.VideoCapture(INPUT_VIDEO_PATH)
    if not cap.isOpened:
        raise IOError("Open video failed!")

    foreground_out = cv.VideoWriter(OUTPUT_FOREGROUND_VIDEO_PATH, cv.VideoWriter_fourcc(*'XVID'), int(cap.get(cv.CAP_PROP_FPS)),
                                (int(cap.get(cv.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))))
    background_out = cv.VideoWriter(OUTPUT_BACKGROUND_VIDEO_PATH, cv.VideoWriter_fourcc(*'XVID'), int(cap.get(cv.CAP_PROP_FPS)),
                                (int(cap.get(cv.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))))
    if not foreground_out.isOpened() or not background_out.isOpened():
        raise IOError("Init videoWriter failed!")

    try:
        for i in tqdm(range(len(foreground_labels))):
            ret, frame = cap.read()
            if not ret:
                raise IOError("Read frame failed!")
                
            label = foreground_labels[i]
            foreground = np.zeros_like(frame)
            foreground.fill(255)
            for (j, k) in label:
                foreground[j][k] = frame[j][k]
                frame[j][k] = [255,255,255]

            foreground_out.write(foreground.copy())
            background_out.write(frame.copy())
            
    except KeyboardInterrupt:
        print('Interrupted!')    

    cap.release()
    foreground_out.release()
    background_out.release()
    cv.destroyAllWindows()

In [ ]:
# generate_foreground_background_videos(foreground_labels)

In [70]:
def get_H_matrix(img1, img2):
    # minimum number of matches we want find between these two images
    MIN_MATCH_COUNT = 10

    # img1 = cv.imread('SAL_images_per_60/SAL_53.jpg',0)          # queryImage
    # img2 = cv.imread('SAL_images_per_60/SAL_83.jpg',0) # trainImage
    # img2_copy = img2.copy()
    # img1_copy = img1.copy()

    # prev, cur are frames
    # initiate feature detector, currently use SIFT, may try orb later
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    # kp1, des1 = sift.detectAndCompute(cv.cvtColor(img1.copy(), cv.COLOR_BGR2GRAY), None)
    # kp2, des2 = sift.detectAndCompute(cv.cvtColor(img2.copy(), cv.COLOR_BGR2GRAY),None)
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        H, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
        # matchesMask = mask.ravel().tolist()
        # h,w = img1.shape
        # pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        # dst = cv.perspectiveTransform(pts,H)
        # img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
    else:
        raise ValueError( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        # matchesMask = None

    # draw_params = dict(matchColor = (0,255,0), # draw matches in green color
    #                singlePointColor = None,
    #                matchesMask = matchesMask, # draw only inliers
    #                flags = 2)
    # img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
    # plt.imshow(img3, 'gray'),plt.show()


    return H

In [ ]:
def get_all_H_matrix_per_step(INPUT_VIDEO_PATH, step = 1):
    # step - 1 is the number of ignored frames between two frames that are used to compute H matrix
    Hs = []
    cap = cv.VideoCapture(INPUT_VIDEO_PATH)
    if not cap.isOpened:
        raise IOError("Open video failed!")

    ret, prev = cap.read()
    if not ret:
        raise IOError("Read frame failed!")

    try:
        for i in tqdm(range(1, int(cap.get(cv.CAP_PROP_FRAME_COUNT)))):
            ret, cur = cap.read()
            if not ret:
                raise IOError("Read frame failed!")
            
            if i % step == 0:
                Hs.append(get_H_matrix(prev, cur))
                prev = cur
            
    except KeyboardInterrupt:
        print('Interrupted!')    

    cap.release()
    cv.destroyAllWindows()
    return Hs

In [6]:
gap = 30

def fill_holes(frames, frame_gap, foreground_labels):

    filled_frames = []

    for i in range(0, len(frames), frame_gap):

        if i + frame_gap < len(frames):
            im_be_filled = frames[i]
            im_later = frames[i + frame_gap]
            im_out = cv.warpPerspective(im_later, get_H_matrix(im_be_filled, im_later), (im_later.shape[1],im_later.shape[0]), flags=cv.WARP_INVERSE_MAP)
            for j, k in foreground_labels[i]:
                im_be_filled[j][k] = im_out[j][k]
        else:
            break
        filled_frames.append(im_be_filled)
        # cv.imshow("cur", im_be_filled)
        # cv.waitKey(0)
    return filled_frames


# filled_frames = fill_holes(read_frames(OUTPUT_BACKGROUND_VIDEO_PATH), gap, foreground_labels)


def fill_holes():
    frames = read_frames(OUTPUT_BACKGROUND_VIDEO_PATH)
    filled_frames = []

    if len(frames) != len(foreground_labels):
        raise EOFError("fasudf")

    for i in tqdm(range(0, len(frames), gap)):
        cur_label = foreground_labels[i].copy()
        last_index = i
        if last_index + gap < len(frames):
        # while len(cur_label) > 0:
        #     print(len(cur_label))
            # im_out = cv.warpPerspective(frames[last_index + gap], get_H_matrix(frames[i], frames[last_index + gap]), \
            #     (frames[i].shape[1], frames[i].shape[0]), flags=cv.WARP_INVERSE_MAP)
            # convert to list due to the need to remove while iterating
            if last_index + gap >= len(frames):
                break
            next_label = foreground_labels[last_index + gap]
            srcs = [list(t) for t in cur_label]
            # point x = j, y = i, so reverse below in a[::-1]
            #perspectivetransform needs 3 dimensions, so we must manually wrap a nonsense dimension to make below 1 * N * 2
            srcpts = np.array([[a[::-1] for a in srcs]]).astype(np.float32)
            H = get_H_matrix(frames[i], frames[last_index + gap])
            dstpts = np.rint(cv.perspectiveTransform(srcpts, H)[0]).astype(int)
            for (x, y), (corresponding_x, corresponding_y) in zip(np.rint(srcpts[0]).astype(int), dstpts):
                if (corresponding_y, corresponding_x) not in next_label and 0 <= corresponding_x < width and 0 <= corresponding_y < height:
                    cur_label.remove((y, x))
                    frames[i][y][x] = frames[last_index + gap][corresponding_y][corresponding_x]
            last_index += gap
        else:
            break
        filled_frames.append(frames[i])

        # else:
        #     while len(cur_label) > 0:
        #         print(len(cur_label))
        #         # im_out = cv.warpPerspective(frames[last_index + gap], get_H_matrix(frames[i], frames[last_index + gap]), \
        #         #     (frames[i].shape[1], frames[i].shape[0]), flags=cv.WARP_INVERSE_MAP)
        #         # convert to list due to the need to remove while iterating
        #         if last_index - gap < 0:
        #             break
        #         next_label = foreground_labels[last_index - gap]
        #         srcs = [list(t) for t in cur_label]
        #         H = get_H_matrix(frames[i], frames[last_index - gap])
        #         for row, col in srcs:
        #             [corresponding_row, corresponding_col, _] = np.matmul(H, np.array([row, col, 1]))
        #             corresponding_row = round(corresponding_row)
        #             corresponding_col = round(corresponding_col)
        #             if (corresponding_row, corresponding_col) not in next_label and 0 <= corresponding_row < height and 0 <= corresponding_col < width:
        #                 cur_label.remove((row, col))
        #                 frames[i][row][col] = frames[last_index - gap][corresponding_row][corresponding_col]
        #         last_index -= gap

        # else:
        #     if i - gap < 0:
        #         raise IOError("Filled Error!")
        #     im_out = cv.warpPerspective(frames[i - gap], Hs[(i - gap) % gap], (im_later.shape[1],im_later.shape[0]))


        # for j, k in foreground_labels[i]:
        #         im_be_filled[j][k] = im_out[j][k]
        # filled_frames.append(im_be_filled)
        # cv.imshow("cur", im_be_filled)
        # cv.waitKey(0)
    return filled_frames


# filled_frames = fill_holes()

 93%|█████████▎| 13/14 [00:35<00:02,  2.75s/it]


In [59]:
def interpolate_foreground_on_panorama(panorama_img, foreground_labels_set):
    # cv.resize(panorama_img, [int(round(panorama_img.shape[1] * 0.5)), int(round(panorama_img.shape[0] * 0.5))], interpolation=cv.INTER_LINEAR_EXACT)
    for i in range(4, len(panorama_img) - 4):
        for j in range(4, len(panorama_img[0]) - 4):
            if panorama_img[i][j][0] == 255:
                continue
            if (i, j) in foreground_labels_set:
                continue
            fore_count = 0
            cur_pix = np.array([0, 0, 0])
            for m in range(i - 2, i + 3):
                for n in range(j - 2, j + 3):
                    if m == i and n == j:
                        continue
                    if (m, n) in foreground_labels_set:
                        fore_count += 1
                        cur_pix = np.add(cur_pix, panorama_img[m][n])
            if fore_count >= 5:
                panorama_img[i][j] = cur_pix / fore_count

    # visited = set()
    # for index in tqdm(foreground_labels_set):
    #     for m in range(index[0] - 1, index[0] + 2):
    #         for n in range(index[1] - 1, index[1] + 2):
    #             if (m ,n) in visited:
    #                 continue
    #             cur_pix = np.array([0, 0, 0])
    #             for a in range(m - 1, m + 2):
    #                 for b in range(n - 1, n + 2):
    #                     cur_pix = np.add(cur_pix, panorama_img[a][b])
    #             panorama_img[m][n] = cur_pix / 15
    #             visited.add((m, n))
    # for i in tqdm(range(4, len(panorama_img) - 4)):
    #     for j in range(4, len(panorama_img[0]) - 4):
    #         if (i, j) not in foreground_labels_set:
    #             continue
    #         for a in range(i - 1, i + 2):
    #             for b in range (j - 1, j + 2):
    #                 if (a, b) in foreground_labels_set:
    #                     continue
    #                 cur_pix = np.array([0, 0, 0])
    #                 for m in range(i - 2, i + 3):
    #                     for n in range(j - 2, j + 3):
    #                         if m == i and n == j:
    #                             continue
    #                         if (m, n) in foreground_labels_set:
    #                             cur_pix = np.add(cur_pix, panorama_img[m][n])
    #                 panorama_img[i][j] = cur_pix / 15
    #                 foreground_labels_set.add((a, b))

In [71]:
def fill_in_foreground_and_generate_img(n):

    cap = cv.VideoCapture(INPUT_VIDEO_PATH)
    if not cap.isOpened:
        raise IOError("Open video failed!")

    panorama = cv.imread(OUTPUT_PANORAMA_IMG_PATH)
    panorama_foreground_set = set()
    panorama_height = len(panorama)
    panorama_width = len(panorama[0])
    try:
        for i in tqdm(range(1, int(cap.get(cv.CAP_PROP_FRAME_COUNT)))):
            ret, prev = cap.read()
            if not ret:
                raise IOError("Read frame failed!")

            if i % n == 0:
                H = get_H_matrix(prev, panorama)
                cur_label = foreground_labels[i].copy()
                srcs = [list(t) for t in cur_label]
                srcpts = np.array([[a[::-1] for a in srcs]]).astype(np.float32)
                dstpts = np.rint(cv.perspectiveTransform(srcpts, H)[0]).astype(int)
                for (x, y), (corresponding_x, corresponding_y) in zip(np.rint(srcpts[0]).astype(int), dstpts):
                    if 0 <= corresponding_x < panorama_width and 0 <= corresponding_y < panorama_height:
                        panorama[corresponding_y][corresponding_x] = prev[y][x]
                        panorama_foreground_set.add((corresponding_y, corresponding_x))

    except KeyboardInterrupt:
        print('Interrupted!')

    interpolate_foreground_on_panorama(panorama, panorama_foreground_set)
        # cv.imshow(str(frame_count), prev)
        # cv.waitKey(0)
        # cv.destroyAllWindows()
        # cv.waitKey(1)


    cv.imshow('panorama', panorama)
    cv.imwrite(os.path.join(OUTPUT_PATH, "{}_panorama_output1.jpg".format(FILENAME.split('.')[0])), panorama)
    cv.waitKey(0)
    cv.destroyAllWindows()
    cv.waitKey(1)


In [11]:
def fill_in_foreground_and_generate_video():

    cap = cv.VideoCapture(INPUT_VIDEO_PATH)
    if not cap.isOpened:
        raise IOError("Open video failed!")

    panorama = cv.imread(OUTPUT_PANORAMA_IMG_PATH)
    panorama_height = len(panorama)
    panorama_width = len(panorama[0])
    panorama_foreground_set = set()
    panorama_video_out = cv.VideoWriter("out/panorama_video.mp4", cv.VideoWriter_fourcc(*'XVID'), int(cap.get(cv.CAP_PROP_FPS)),
                                (int(panorama_width), int(panorama_height)))
    if not panorama_video_out.isOpened():
        raise IOError("Init videoWriter failed!")
    try:
        for i in tqdm(range(0, int(cap.get(cv.CAP_PROP_FRAME_COUNT)))):
            ret, prev = cap.read()
            if not ret:
                raise IOError("Read frame failed!")

            panorama_foreground_set.clear()
            cur_panorama = panorama.copy()
            H = get_H_matrix(prev, cur_panorama)
            cur_label = foreground_labels[i].copy()
            srcs = [list(t) for t in cur_label]
            srcpts = np.array([[a[::-1] for a in srcs]]).astype(np.float32)
            dstpts = np.rint(cv.perspectiveTransform(srcpts, H)[0]).astype(int)
            for (x, y), (corresponding_x, corresponding_y) in zip(np.rint(srcpts[0]).astype(int), dstpts):
                if 0 <= corresponding_x < panorama_width and 0 <= corresponding_y < panorama_height:
                    cur_panorama[corresponding_y][corresponding_x] = prev[y][x]
                    panorama_foreground_set.add((corresponding_y, corresponding_x))

            interpolate_foreground_on_panorama(cur_panorama, panorama_foreground_set)
            panorama_video_out.write(cur_panorama)
    except KeyboardInterrupt:
        print('Interrupted!')


        # cv.imshow(str(frame_count), prev)
        # cv.waitKey(0)
        # cv.destroyAllWindows()
        # cv.waitKey(1)

    cap.release()
    panorama_video_out.release()

    cv.destroyAllWindows()
    cv.waitKey(1)


In [64]:
def fill_in_foreground_and_generate_all_imgs(output_folder, start, end):
    cap = cv.VideoCapture(INPUT_VIDEO_PATH)

    if not cap.isOpened:
        raise IOError("Open video failed!")
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    panorama = cv.imread(OUTPUT_PANORAMA_IMG_PATH)
    panorama_height = len(panorama)
    panorama_width = len(panorama[0])
    panorama_foreground_set = set()
    try:
        if start < 0:
            start = 0
        if start >= int(cap.get(cv.CAP_PROP_FRAME_COUNT)):
            return
        if end > int(cap.get(cv.CAP_PROP_FRAME_COUNT)):
            end = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
        for i in tqdm(range(0, int(cap.get(cv.CAP_PROP_FRAME_COUNT)))):
            ret, prev = cap.read()
            if not start <= i < end:
                continue
            # if not ret:
            #     raise IOError("Read frame failed!")

            panorama_foreground_set.clear()
            cur_panorama = panorama.copy()
            H = get_H_matrix(prev, cur_panorama)
            cur_label = foreground_labels[i].copy()
            srcs = [list(t) for t in cur_label]
            if len(srcs) != 0:
                srcpts = np.array([[a[::-1] for a in srcs]]).astype(np.float32)
                dstpts = np.rint(cv.perspectiveTransform(srcpts, H)[0]).astype(int)
                for (x, y), (corresponding_x, corresponding_y) in zip(np.rint(srcpts[0]).astype(int), dstpts):
                    if 0 <= corresponding_x < panorama_width and 0 <= corresponding_y < panorama_height:
                        cur_panorama[corresponding_y][corresponding_x] = prev[y][x]
                        panorama_foreground_set.add((corresponding_y, corresponding_x))

                interpolate_foreground_on_panorama(cur_panorama, panorama_foreground_set)
            cv.imwrite(f"{output_folder}/{FILENAME.split('.')[0]}_panorama_with_foreground_{i}.jpg", cur_panorama)
    except KeyboardInterrupt:
        print('Interrupted!')


    cap.release()


In [77]:
fill_in_foreground_and_generate_all_imgs("output3_best", 325, 350)

 58%|█████▊    | 344/595 [1:12:44<53:04, 12.69s/it]    

Interrupted!


In [31]:
def create_app3_pre_video_from_img(img_folder):
    if not os.path.exists(img_folder):
        raise IOError("Open folder failed!")
    cap = cv.VideoCapture(INPUT_VIDEO_PATH)

    panorama = cv.imread(OUTPUT_PANORAMA_IMG_PATH)
    panorama_height = len(panorama)
    panorama_width = len(panorama[0])
    panorama_video_out = cv.VideoWriter(f"out/{FILENAME.split('.')[0]}_application_2_pre_video.mp4", cv.VideoWriter_fourcc(*'XVID'), int(cap.get(cv.CAP_PROP_FPS)), (int(panorama_width), int(panorama_height)))

    try:
        for i in tqdm(range(0, int(cap.get(cv.CAP_PROP_FRAME_COUNT)))):
            cur_img_name = f"{img_folder}/{FILENAME.split('.')[0]}_panorama_with_foreground_{i}.jpg"
            if os.path.isfile(cur_img_name):
                panorama_video_out.write(cv.imread(cur_img_name))
    except KeyboardInterrupt:
        print('Interrupted!')
    cap.release()
    panorama_video_out.release()

In [32]:
create_app3_pre_video_from_img("output3")

100%|██████████| 595/595 [00:40<00:00, 14.82it/s]
